In [ ]:
import numpy as np
import pandas as pd
import matplotlib
from gpcam.gp_optimizer import GPOptimizer
import matplotlib.pyplot as plt
import mpltern
from matplotlib.patches import ArrowStyle, FancyArrowPatch
from gpcam import fvGPOptimizer

In [ ]:
def add_arrows_to_ternary_plot(label_top, label_left, label_right, ax=None):
    ax = ax or plt.gca()
    arrowstyle = ArrowStyle("simple", head_length=10, head_width=5)
    kwargs_arrow = {
        "transform": ax.transAxes,  # Used with ``ax.transAxesProjection``
        "arrowstyle": arrowstyle,
        "linewidth": 1,
        "clip_on": False,  # To plot arrows outside triangle
        "zorder": -10,  # Very low value not to hide e.g. tick labels.
    }

    # Start of arrows in barycentric coordinates.
    ta = np.array([0.0, -0.1, 1.1])
    la = np.array([1.1, 0.0, -0.1])
    ra = np.array([-0.1, 1.1, 0.0])

    # End of arrows in barycentric coordinates.
    tb = np.array([1.0, -0.1, 0.1])
    lb = np.array([0.1, 1.0, -0.1])
    rb = np.array([-0.1, 0.1, 1.0])

    # This transforms the above barycentric coordinates to the original Axes
    # coordinates. In combination with ``ax.transAxes``, we can plot arrows fixed
    # to the Axes coordinates.
    f = ax.transAxesProjection.transform

    tarrow = FancyArrowPatch(f(ta), f(tb), ec="k", fc="k", **kwargs_arrow)
    larrow = FancyArrowPatch(f(la), f(lb), ec="k", fc="k", **kwargs_arrow)
    rarrow = FancyArrowPatch(f(ra), f(rb), ec="k", fc="k", **kwargs_arrow)
    ax.add_patch(tarrow)
    ax.add_patch(larrow)
    ax.add_patch(rarrow)

    # To put the axis-labels at the positions consistent with the arrows above, it
    # may be better to put the axis-label-text directly as follows rather than
    # using e.g.  ax.set_tlabel.
    kwargs_label = {
        "transform": ax.transTernaryAxes,
        "backgroundcolor": "w",
        "ha": "center",
        "va": "center",
        "rotation_mode": "anchor",
        "zorder": -9,  # A bit higher on arrows, but still lower than others.
    }

    # Put axis-labels on the midpoints of arrows.
    tpos = (ta + tb) * 0.5
    lpos = (la + lb) * 0.5
    rpos = (ra + rb) * 0.5

    ax.text(*tpos, label_top, c="k", rotation=-60, **kwargs_label)
    ax.text(*lpos, label_left, c="k", rotation=60, **kwargs_label)
    ax.text(*rpos, label_right, c="k", rotation=0, **kwargs_label)

In [ ]:
df = pd.read_csv(
    "/Users/wiebke/Documents/Data/saxs-waxs-samples/BCP_Mixing/2024_10_17-23_58k-34k_mixture/2024_10_17-23_58k-34k_mixture.csv"
)
additive = df["Fraction Additive"].values
s2vp34k = df["Fraction 34k"].values
s2vp58k = df["Fraction 58k"].values
fwhms = df["Fwhm"].values
peak_positions = df["Peak Position"].values

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(18, 6), subplot_kw={"projection": "ternary"})

material_top = "Additive"
material_left = "S2VP58k"
material_right = "S2VP34k"

# Plot 1: Colored by s2vp35k_ratio
cmap = matplotlib.colormaps.get_cmap("viridis")
cmap.set_bad(color="red")
sc1 = axs[0].scatter(
    additive,
    s2vp58k,
    s2vp34k,
    c=peak_positions,
    cmap=cmap,
    vmin=0.1 * 2 * np.pi / 42,  # 42nm domain spacing
    vmax=0.1 * 2 * np.pi / 25,  # 25nm domain spacing
)
axs[0].set_title(f"Peak position")
add_arrows_to_ternary_plot(
    f"{material_top} (%)", f"{material_left} (%)", f"{material_right} (%)", ax=axs[0]
)
fig.colorbar(sc1, ax=axs[0], label=f"peak position", shrink=0.5)
axs[0].grid()

# Plot 2: Colored by fwhm
cmap = matplotlib.colormaps.get_cmap("inferno")
cmap.set_bad(color="green")
sc2 = axs[1].scatter(
    additive, s2vp58k, s2vp34k, c=fwhms, cmap="inferno", vmin=0.002, vmax=0.006
)
axs[1].set_title(f"Fwhm")
add_arrows_to_ternary_plot(
    f"{material_top} (%)", f"{material_left} (%)", f"{material_right} (%)", ax=axs[1]
)
fig.colorbar(sc2, ax=axs[1], label="fwhm", shrink=0.5)
axs[1].grid()

In [ ]:
x_data = np.stack([s2vp58k, s2vp34k], axis=1)
x_data.shape
q_target = 0.1 * 2 * np.pi / 30
peak_pos_min = 0.1 * 2 * np.pi / 42
peak_pos_max = 0.1 * 2 * np.pi / 25
max_peak_dist = max(np.abs(peak_pos_min - q_target), np.abs(peak_pos_max - q_target))
y_data_peaks_only = abs(peak_positions - q_target)
y_data_fwhm_only = fwhms
y_data_mixed = 0.2 * abs(y_data_peaks_only) / (max_peak_dist) + 0.8 * (
    y_data_fwhm_only / (0.006 - 0.003) - 0.003
)

In [ ]:
y_data = np.hstack(
    [peak_positions.reshape(len(peak_positions), 1), fwhms.reshape(len(fwhms), 1)]
)

In [ ]:
from gpcam.gp_kernels import *


def kernel(x1, x2, hps):
    ds = get_anisotropic_distance_matrix(x1[:, 0:2], x2[:, 0:2], hps[1:3])
    dt = get_distance_matrix(x1[:, 2:3], x2[:, 2:3])
    k = hps[0] * matern_kernel_diff1(ds, 1.0) * exponential_kernel(dt, hps[3])
    return k


# bounds of the input space (parameters that are controlled)
bounds = np.array([[0, 1], [0, 1]])
# Set up hyperparameters for kernel
hps_bounds = np.array([[0.0001, 10], [0.0001, 10], [0.0001, 10], [0.0001, 100.0]])
# initialize the GPOptimizer
init_hps = np.random.uniform(
    size=len(hps_bounds), low=hps_bounds[:, 0], high=hps_bounds[:, 1]
)
my_gpo = fvGPOptimizer(
    x_data, y_data, init_hyperparameters=init_hps, gp_kernel_function=kernel
)

my_gpo.train(hyperparameter_bounds=hps_bounds, max_iter=10000, method="mcmc")
print("hyperparameters after 1st training: ", my_gpo.hyperparameters)

In [ ]:
x_pred = np.random.dirichlet([1, 1, 1], size=500)
x_pred_reduced = x_pred[:, :2]
print(x_pred_reduced.shape)
mean_calulation = my_gpo.posterior_mean(x_pred_reduced, x_out=np.array([0, 1]))
mean1 = mean_calulation["f(x)"][:, 0]
mean2 = mean_calulation["f(x)"][:, 1]
variance_calculation = my_gpo.posterior_covariance(
    x_pred_reduced, x_out=np.array([0, 1])
)
covariance1 = variance_calculation["v(x)"][:, 0]
covariance2 = variance_calculation["v(x)"][:, 1]

In [ ]:
mean = mean1
covariance = covariance1


fig, axs = plt.subplots(1, 3, figsize=(18, 6), subplot_kw={"projection": "ternary"})

sc1 = axs[0].scatter(
    additive,
    s2vp58k,
    s2vp34k,
    c=y_data_peaks_only,
    cmap="viridis",
    vmin=-max_peak_dist,
    vmax=max_peak_dist,
)
add_arrows_to_ternary_plot(
    f"{material_top} (%)", f"{material_left} (%)", f"{material_right} (%)", ax=axs[0]
)
fig.colorbar(sc1, ax=axs[0], label=f"distance from peak position", shrink=0.5)
axs[0].grid()

sc2 = axs[1].tripcolor(
    x_pred[:, 2],
    x_pred[:, 0],
    x_pred[:, 1],
    mean,
    shading="gouraud",
    rasterized=True,
    vmin=np.min(mean),
    vmax=np.max(mean),
)
add_arrows_to_ternary_plot(
    f"{material_top} (%)", f"{material_left} (%)", f"{material_right} (%)", ax=axs[0]
)
fig.colorbar(sc2, ax=axs[1], label=f"gp mean", shrink=0.5)
axs[1].grid()

sc3 = axs[2].tripcolor(
    x_pred[:, 2],
    x_pred[:, 0],
    x_pred[:, 1],
    covariance,
    cmap="inferno",
    shading="gouraud",
    rasterized=True,
)
add_arrows_to_ternary_plot(
    f"{material_top} (%)", f"{material_left} (%)", f"{material_right} (%)", ax=axs[0]
)
fig.colorbar(sc3, ax=axs[2], label=f"gp covariance", shrink=0.5)

In [ ]:
mean = mean2
covariance = covariance2


fig, axs = plt.subplots(1, 3, figsize=(18, 6), subplot_kw={"projection": "ternary"})

sc1 = axs[0].scatter(
    additive,
    s2vp58k,
    s2vp34k,
    c=y_data_fwhm_only,
    cmap="viridis",
    # vmin=-max_peak_dist,
    # vmax=max_peak_dist,
)
add_arrows_to_ternary_plot(
    f"{material_top} (%)", f"{material_left} (%)", f"{material_right} (%)", ax=axs[0]
)
fig.colorbar(sc1, ax=axs[0], label=f"distance from peak position", shrink=0.5)
axs[0].grid()

sc2 = axs[1].tripcolor(
    x_pred[:, 2],
    x_pred[:, 0],
    x_pred[:, 1],
    mean,
    shading="gouraud",
    rasterized=True,
    vmin=np.min(mean),
    vmax=np.max(mean),
)
add_arrows_to_ternary_plot(
    f"{material_top} (%)", f"{material_left} (%)", f"{material_right} (%)", ax=axs[0]
)
fig.colorbar(sc2, ax=axs[1], label=f"gp mean", shrink=0.5)
axs[1].grid()

sc3 = axs[2].tripcolor(
    x_pred[:, 2],
    x_pred[:, 0],
    x_pred[:, 1],
    covariance,
    cmap="inferno",
    shading="gouraud",
    rasterized=True,
)
add_arrows_to_ternary_plot(
    f"{material_top} (%)", f"{material_left} (%)", f"{material_right} (%)", ax=axs[0]
)
fig.colorbar(sc3, ax=axs[2], label=f"gp covariance", shrink=0.5)

In [ ]:
from scipy.optimize import LinearConstraint

y_data_peaks_only = abs(peak_positions - q_target)
y_data_fwhm_only = fwhms
y_data_mixed = 0.2 * abs(y_data_peaks_only) / (max_peak_dist) + 0.8 * (
    y_data_fwhm_only / (0.006 - 0.003) - 0.003
)


# Target for both peak position and fwhm
def acq_func1(x, obj):
    # 3.0 for 95 percent confidence interval
    mean = obj.posterior_mean(x, x_out=np.array([0, 1]))["f(x)"]
    mean = -abs(mean - [q_target, 0.002])
    cov = obj.posterior_covariance(x, x_out=np.array([0, 1]))["v(x)"]
    v = mean * np.sqrt(cov)
    return np.sum(v.reshape(len(x), len(x_out), order="F"), axis=1)


# Uncertainty only
def acq_func2(x, obj):
    a = 3.0  # 3.0 for 95 percent confidence interval
    mean = obj.posterior_mean(x, x_out=np.array([0, 1]))["f(x)"]
    cov = obj.posterior_covariance(x, x_out=np.array([0, 1]))["v(x)"]
    v = np.sqrt(cov)
    return np.sum(v.reshape(len(x), len(x_out), order="F"), axis=1)


A = np.array([1, 1])
lc = LinearConstraint(A, 0.2, 1.0)

df = pd.DataFrame(
    columns=[
        "Fraction 58k",
        "Fraction 34k",
        "Fraction Additive",
        "Swell Ratio",
        "Expected Peak Position Distance",
    ]
)

acq_func = [acq_func1, acq_func2]

In [ ]:
x_out = np.array([0, 1])
for n in range(1, 10):
    new_x_peaks = my_gpo.ask(
        bounds,
        x_out,
        n=1,
        acquisition_function=acq_func[(n - 1) % 2],
        constraints=(lc,),
    )
    print(new_x_peaks)
    new_row = pd.DataFrame(
        {
            "Fraction 58k": [new_x_peaks["x"][0][0]],
            "Fraction 34k": [new_x_peaks["x"][0][1]],
            "Expected Peak Position Distance": [new_x_peaks["f(x)"][0]],
        },
    )
    df = pd.concat([df, new_row], ignore_index=True)

df["Fraction Additive"] = 1 - df["Fraction 58k"] - df["Fraction 34k"]
df["Swell Ratio"] = 1 + df["Fraction Additive"] / (
    df["Fraction 34k"] + df["Fraction 58k"]
)
print(df)
df.to_csv("Suggested_peak_pos_2_58k-34k.csv")